## Skapa en geojson av SAT leden
* issue [#181](https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/181)
* denna [Notebook](https://github.com/salgo60/Stockholm_Archipelago_Trail/tree/main/notebook/181_geojson_SAT_trail)

In [1]:
import time
import datetime  
start_time = time.time()
start_str = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
print(f"Started: {start_str}")


Started: 2025-09-22 07:25


In [2]:
import requests, json
from lxml import etree
from shapely.geometry import LineString
from shapely.ops import unary_union
from collections import Counter, defaultdict
from requests.adapters import HTTPAdapter, Retry
from shapely.geometry import mapping
from pathlib import Path

OSM_REL = 19012437  # SAT superrelation
EXCLUDE_ROLES = {"alternate", "detour", "connection"}  # justera vid behov

# ---------- Nätverk: session med retry + no-cache ----------
def make_session():
    s = requests.Session()
    retries = Retry(total=5, backoff_factor=1.2, status_forcelist=[429, 500, 502, 503, 504])
    s.mount("https://", HTTPAdapter(max_retries=retries))
    return s

SESSION = make_session()

def fetch_relation_full_with_headers(rel_id: int, timeout=60, session=SESSION):
    url = f"https://api.openstreetmap.org/api/0.6/relation/{rel_id}/full"
    r = session.get(
        url,
        timeout=timeout,
        headers={
            "Cache-Control": "no-cache",
            "Pragma": "no-cache",
            "User-Agent": "sat-debug/0.1 (+contact)"
        },
    )
    r.raise_for_status()
    root = etree.fromstring(r.content)
    return root, r.headers

# ---------- Medlems-hjälpare ----------
def relation_members(xml_root, rel_id: int):
    """Returnerar medlemmar för relationen rel_id som dict:
       {"way": [(id, role), ...], "relation": [(id, role), ...], "node": [(id, role), ...]}"""
    members = defaultdict(list)
    for rel in xml_root.findall("relation"):
        if rel.attrib["id"] == str(rel_id):
            for m in rel.findall("member"):
                mtype = m.attrib.get("type")
                mid = int(m.attrib.get("ref"))
                role = m.attrib.get("role", "")
                members[mtype].append((mid, role))
            break
    return members

def summarize_members(members, title=""):
    if title:
        print(f"== {title} ==")
    for t in ("relation", "way", "node"):
        lst = members.get(t, [])
        if not lst:
            continue
        roles = [r for _, r in lst]
        print(f"{t}: {len(lst)} st | roller: {dict(Counter(roles))}")

def member_relation_ids(xml_root, rel_id: int):
    ids = set()
    for rel in xml_root.findall("relation"):
        if rel.attrib["id"] == str(rel_id):
            for m in rel.findall("member"):
                if m.attrib.get("type") == "relation":
                    ids.add(int(m.attrib["ref"]))
            break
    return ids

def member_way_ids(xml_root, rel_id: int, exclude_roles=EXCLUDE_ROLES):
    ids = set()
    for rel in xml_root.findall("relation"):
        if rel.attrib["id"] == str(rel_id):
            for m in rel.findall("member"):
                if m.attrib.get("type") == "way":
                    role = m.attrib.get("role", "")
                    if role not in exclude_roles:
                        ids.add(m.attrib["ref"])
            break
    return ids

# ---------- Ways -> koordinater, men endast för medlemmar ----------
def ways_by_id_from_xml(xml_root, allowed_way_ids: set[str]):
    # Bygg upp noder
    nodes = {n.attrib["id"]: (float(n.attrib["lon"]), float(n.attrib["lat"]))
             for n in xml_root.findall("node")}
    # Plocka ut enbart de ways vi vill ha
    ways = {}
    for w in xml_root.findall("way"):
        wid = w.attrib["id"]
        if wid not in allowed_way_ids:
            continue
        coords = []
        for nd in w.findall("nd"):
            ref = nd.attrib["ref"]
            if ref in nodes:
                coords.append(nodes[ref])
        if len(coords) > 1:
            ways[wid] = coords
    return ways

# ---------- Körning ----------
# 1) Färsk hämtning av superrelationen
root_super, hdr = fetch_relation_full_with_headers(OSM_REL)
print("HTTP Date:", hdr.get("Date"))
print("ETag:", hdr.get("ETag"))
print("Last-Modified:", hdr.get("Last-Modified"))

# Metadata från själva relationen
rel_version = rel_ts = None
for rel in root_super.findall("relation"):
    if rel.attrib["id"] == str(OSM_REL):
        rel_version = rel.attrib.get("version")
        rel_ts = rel.attrib.get("timestamp")
        break
print(f"Relation {OSM_REL} version: {rel_version} | timestamp: {rel_ts}")

# 2) Medlemsöversikt (superrelationen)
members_super = relation_members(root_super, OSM_REL)
summarize_members(members_super, title=f"Relation {OSM_REL}")

# 3) Jämför refererade delrelationer vs faktiskt närvarande relation-element
ref_child_ids = member_relation_ids(root_super, OSM_REL)

present_child_ids = {int(r.attrib["id"])
                     for r in root_super.findall("relation")
                     if r.attrib["id"] != str(OSM_REL)}

missing_ids = sorted(ref_child_ids - present_child_ids)
extra_ids = sorted(present_child_ids - ref_child_ids)

print(f"Delrelationer (referenser): {len(ref_child_ids)}")
print(f"Delrelationer (hämtade element): {len(present_child_ids)}")
print("Saknas som element (refererad men ej närvarande i /full):", missing_ids)
print("Överflödiga element (närvarande men ej refererad):", extra_ids)

# 4) Bygg ways från superrelationens egna ways
super_way_ids = member_way_ids(root_super, OSM_REL)
all_ways = ways_by_id_from_xml(root_super, super_way_ids)

# 5) Hämta ways från varje delrelation (referenserna används; fetchas separat)
failed_children = []
for rid in sorted(ref_child_ids):
    try:
        rxml, _ = fetch_relation_full_with_headers(rid)
        sub_way_ids = member_way_ids(rxml, rid)
        sub_ways = ways_by_id_from_xml(rxml, sub_way_ids)
        all_ways.update(sub_ways)  # dedup på way-id
    except requests.HTTPError as e:
        failed_children.append((rid, str(e)))

if failed_children:
    print("Varning: kunde inte hämta följande delrelationer:")
    for rid, err in failed_children:
        print(f"  - {rid}: {err}")

# 6) Shapely geometrier
lines = [LineString(coords) for coords in all_ways.values() if len(coords) > 1]
if not lines:
    raise RuntimeError("Hittade inga linjer – avbryter.")
trail = unary_union(lines)
center_lat, center_lon = trail.centroid.y, trail.centroid.x

print(
    f"Delrelationer (ref): {len(ref_child_ids)} | "
    f"Delrelationer (hämtade): {len(present_child_ids)} | "
    f"Ways: {len(all_ways)} | Linjesegment: {len(lines)} | "
    f"Centroid: {center_lat:.6f},{center_lon:.6f}"
)

# 7) Spara som GeoJSON (flattenade LineStrings)
features = []
if trail.geom_type == "LineString":
    features = [{"type": "Feature", "geometry": mapping(trail), "properties": {}}]
elif trail.geom_type == "MultiLineString":
    for seg in trail.geoms:
        features.append({"type": "Feature", "geometry": mapping(seg), "properties": {}})
else:
    for seg in getattr(trail, "geoms", []):
        if seg.geom_type == "LineString":
            features.append({"type": "Feature", "geometry": mapping(seg), "properties": {}})
        elif seg.geom_type == "MultiLineString":
            for s in seg.geoms:
                features.append({"type": "Feature", "geometry": mapping(s), "properties": {}})

gj = {"type": "FeatureCollection", "features": features}
Path("SAT_full.geojson").write_text(json.dumps(gj), encoding="utf-8")
print("Sparade: SAT_full.geojson")


HTTP Date: Mon, 22 Sep 2025 05:25:10 GMT
ETag: None
Last-Modified: None
Relation 19012437 version: 31 | timestamp: 2025-09-22T05:04:08Z
== Relation 19012437 ==
relation: 20 st | roller: {'': 20}
Delrelationer (referenser): 20
Delrelationer (hämtade element): 20
Saknas som element (refererad men ej närvarande i /full): []
Överflödiga element (närvarande men ej refererad): []
Delrelationer (ref): 20 | Delrelationer (hämtade): 20 | Ways: 754 | Linjesegment: 754 | Centroid: 59.274339,18.608028
Sparade: SAT_full.geojson


In [3]:
# fresh fetch every time (don’t reuse root_super)
root_super, hdr = fetch_relation_full_with_headers(OSM_REL)
print("HTTP Date:", hdr.get("Date"))
print("ETag:", hdr.get("ETag"))
print("Last-Modified:", hdr.get("Last-Modified"))



HTTP Date: Mon, 22 Sep 2025 05:25:12 GMT
ETag: None
Last-Modified: None


In [4]:
# IDs som superrelationen REFERERAR till (relation-medlemmar)
refs = []
for rel in root_super.findall("relation"):
    if rel.attrib["id"] == str(OSM_REL):
        for m in rel.findall("member"):
            if m.attrib.get("type") == "relation":
                refs.append(int(m.attrib["ref"]))
        break
refs_set = set(refs)

# IDs som faktiskt finns som <relation>-ELEMENT i /full-svaret (exkl. superrelationen själv)
present_set = {int(r.attrib["id"]) for r in root_super.findall("relation") if r.attrib["id"] != str(OSM_REL)}

print("Antal referenser:", len(refs_set))
print("Antal relation-element:", len(present_set))
print("Saknas som element (finns som referens):", sorted(refs_set - present_set))
print("Överflödiga element (inte listade som medlem):", sorted(present_set - refs_set))


Antal referenser: 20
Antal relation-element: 20
Saknas som element (finns som referens): []
Överflödiga element (inte listade som medlem): []


In [5]:
for rel in root_super.findall("relation"):
    if rel.attrib["id"] == str(OSM_REL):
        print("Relation version:", rel.attrib.get("version"), "timestamp:", rel.attrib.get("timestamp"))
        break


Relation version: 31 timestamp: 2025-09-22T05:04:08Z


In [6]:
    end_time = time.time()
    duration = end_time - start_time
    print(f"Finished in {duration:.2f} seconds.")


Finished in 2.14 seconds.
